In [ ]:
import psycopg2
import pandas as pd
import time
import os
from datetime import datetime

# ===== CONFIGURATION =====
PG_HOST = "172.16.90.18"
PG_DATABASE = "postgres"
PG_USER = "postgres"
PG_PASSWORD = "admin123"
PG_PORT = 5432

TABLE_NAME = "ecommerce_sales"
LOG_TABLE = "ecommerce_sales_changes"
LAST_RUN_FILE = "last_run_timestamp.txt"

GOOGLE_DRIVE_PATH = r"D:\Google Drive\ETL\ecommerce_sales_sheet.xlsx"
POLL_INTERVAL = 10  # seconds

# ===== HELPER FUNCTIONS =====
def get_last_run_time():
    if os.path.exists(LAST_RUN_FILE):
        with open(LAST_RUN_FILE, "r") as f:
            return f.read().strip()
    return None

def set_last_run_time(timestamp):
    with open(LAST_RUN_FILE, "w") as f:
        f.write(timestamp)

def check_for_changes(conn, last_run):
    with conn.cursor() as cur:
        if last_run:
            cur.execute(f"""
                SELECT MAX(change_time) 
                FROM {LOG_TABLE}
                WHERE change_time > %s;
            """, (last_run,))
        else:
            cur.execute(f"SELECT MAX(change_time) FROM {LOG_TABLE};")
        latest_timestamp = cur.fetchone()[0]
        return latest_timestamp

def export_to_excel(conn):
    query = f"SELECT * FROM {TABLE_NAME};"
    df = pd.read_sql(query, conn)
    df.to_excel(GOOGLE_DRIVE_PATH, index=False)
    print(f"[{datetime.now()}] Exported {len(df)} rows to {GOOGLE_DRIVE_PATH}")

# ===== MAIN LOOP =====
def main():
    print("Starting Trigger-based PG → Google Drive ETL process...")
    last_run = get_last_run_time()

    while True:
        try:
            conn = psycopg2.connect(
                host=PG_HOST, database=PG_DATABASE, user=PG_USER,
                password=PG_PASSWORD, port=PG_PORT
            )
            latest_timestamp = check_for_changes(conn, last_run)

            if latest_timestamp and (not last_run or str(latest_timestamp) > last_run):
                export_to_excel(conn)  # Always overwrite with current live data
                set_last_run_time(str(latest_timestamp))
                last_run = str(latest_timestamp)
            else:
                print(f"[{datetime.now()}] No new changes detected.")

            conn.close()
        except Exception as e:
            print("Error:", e)

        time.sleep(POLL_INTERVAL)

if __name__ == "__main__":
    main()


Starting Trigger-based PG → Google Drive ETL process...
[2025-08-12 10:23:03.090289] No new changes detected.
[2025-08-12 10:23:13.411475] No new changes detected.
[2025-08-12 10:23:23.649440] No new changes detected.
[2025-08-12 10:23:33.761046] No new changes detected.
[2025-08-12 10:23:43.917635] No new changes detected.
[2025-08-12 10:23:54.055828] No new changes detected.
[2025-08-12 10:24:04.115977] No new changes detected.
[2025-08-12 10:24:14.295977] No new changes detected.
[2025-08-12 10:24:24.380611] No new changes detected.
[2025-08-12 10:24:34.626790] No new changes detected.


C:\Users\SLUser.5CD41683X7\AppData\Local\Temp\ipykernel_6412\3808784994.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


[2025-08-12 10:24:46.478705] Exported 1 rows to D:\Google Drive\ETL\ecommerce_sales_sheet.xlsx
[2025-08-12 10:24:56.595903] No new changes detected.
[2025-08-12 10:25:06.711610] No new changes detected.
[2025-08-12 10:25:16.776575] No new changes detected.
[2025-08-12 10:25:26.976874] No new changes detected.
[2025-08-12 10:25:37.042053] No new changes detected.
[2025-08-12 10:25:47.209775] No new changes detected.
[2025-08-12 10:25:57.496231] No new changes detected.
[2025-08-12 10:26:07.572371] No new changes detected.
[2025-08-12 10:26:17.793241] No new changes detected.
[2025-08-12 10:26:27.956999] No new changes detected.
[2025-08-12 10:26:38.048763] No new changes detected.
[2025-08-12 10:26:48.311978] No new changes detected.
[2025-08-12 10:26:58.525769] No new changes detected.
[2025-08-12 10:27:08.626061] No new changes detected.
[2025-08-12 10:27:18.688427] No new changes detected.
[2025-08-12 10:27:28.967141] No new changes detected.
[2025-08-12 10:27:39.279516] No new chang